# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [53]:
# your code goes here
def clean_data(df):
    df = handle_missing_data(df)
    df = drop_duplicates(df)
    df = standardize_column_names(df)
    df = clean_column_formats(df)
    return df

def handle_missing_data(df):
    df.fillna(df.mean(numeric_only=True), inplace=True)
    return df

def drop_duplicates(df):
    df.drop_duplicates(inplace=True)
    return df

def standardize_column_names(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    return df

def clean_column_formats(df):
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].str.strip()
    return df
    
import pandas as pd
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)
clean_df = clean_data(df)
filtered_df = clean_df[(clean_df["total_claim_amount"] > 1000) & (clean_df["response"] == "Yes")]
filtered_yes_response_df = clean_df[clean_df["response"] == "Yes"]
grouped_mean = filtered_yes_response_df.groupby(["policy_type", "gender"])["total_claim_amount"].mean()
#Femela hace more Corporate Auto and special auto than men, and both policy_type have a bigger gap that personal auto
print(grouped_mean)
total_policies_df = clean_df.groupby("state").sum()
clean_df.head()

customer_count_by_state = clean_df.groupby("state").size()
filtered_states = customer_count_by_state[customer_count_by_state > 500]
print(filtered_states)

clean_df.head() 
customer_lifetime_df = clean_df.groupby(["education", "gender"])["total_claim_amount"].agg([ "max", "min", "median"])
print(customer_lifetime_df)

#Males generally have higher maximum and median claim amounts than Females across most education levels, except in some categories like Bachelor's and College, where Females show higher maximum values.
#Females tend to have more variability in their claim amounts, with a larger difference between max and median, especially in the Bachelor's and High School or Below categories.
#Overall, the claim amounts for both genders show significant outliers, particularly in the maximum values, indicating some individuals have exceptionally high claims.

policy_type     gender
Corporate Auto  F         433.738499
                M         408.582459
Personal Auto   F         452.965929
                M         457.010178
Special Auto    F         453.280164
                M         429.527942
Name: total_claim_amount, dtype: float64
state
Arizona       1937
California    3552
Nevada         993
Oregon        2909
Washington     888
dtype: int64
                                     max       min      median
education            gender                                   
Bachelor             F       2156.011022  2.016239  364.800000
                     M       2893.239678  0.382107  377.354191
College              F       2327.166394  0.423310  345.600000
                     M       2275.265075  4.238626  400.181882
Doctor               F       1286.400000  0.517753  331.200000
                     M       1337.063487  0.887629  330.138332
High School or Below F       2452.894264  1.332349  427.200000
                     M       2759